In [ ]:
import sys

sys.path.append("..")

In [2]:
import nest_asyncio
nest_asyncio.apply()

In [3]:
from tqdm import trange

In [4]:
from environment.qlearning.obstacle_avoidance_env import ObstacleAvoidanceEnv
from utils.reader import get_yaml_path, read_file

In [5]:
from agent.qagent import QAgent
from training.qlearning import QLearning

In [6]:
import pygame
import numpy as np

<frozen importlib._bootstrap>:488: RuntimeWarning: Your system is avx2 capable but pygame was not built with support for it. The performance of some of your blits could be adversely affected. Consider enabling compile time detection with environment variables like PYGAME_DETECT_AVX2=1 if you are compiling without cross compilation.
/nix/store/wyq2941n10p3ybrc6n759dhcjyqm0hki-python3-3.13.8-env/lib/python3.13/site-packages/pygame/pkgdata.py:25: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import resource_stream, resource_exists


In [7]:
server_address = "localhost:50051"
client_name = "RLClient"
env = ObstacleAvoidanceEnv(server_address, client_name)
env.connect_to_client()

2025-10-28 11:56:33,950 — INFO — ✓ Connected to localhost:50051



In [8]:
config_path = get_yaml_path("resources", "configurations", "obstacle-avoidance.yml")
config = read_file(config_path)
# print(config)

In [9]:
env.init(config)

2025-10-28 11:56:33,968 — INFO — ✓ Initialization successful


In [10]:
episodes = 100
steps = 5000

In [11]:
agent = QAgent(env, episodes = episodes)
agentId = "00000000-0000-0000-0000-000000000001"
agents = { agentId: agent }

In [12]:
def run_episodes(episode_count, episode_max_steps, render = False):
    running = True
    
    if render:
        pygame.init()
        screen = pygame.display.set_mode((800, 600))
        pygame.display.set_caption(f"Q-Learning obstacle avoidance")
        clock = pygame.time.Clock()
    for _ep in trange(episode_count, desc="Training", unit="ep"):
        obs, _ = env.reset()
        done = False
        total_reward = { agentId: 0 }
        episode_history = []
        step_count = 0
        while not done and step_count < episode_max_steps:
            actions = { k: agents[k].choose_action(v, epsilon_greedy = not render) for k, v in obs.items() }
            next_obs, rewards, terminateds, truncateds, _ = env.step(actions)
            # TODO: take other agents into consideration
            done = terminateds[agentId] or truncateds[agentId]
            if not render:
                for k, _ in next_obs.items():
                    agents[k].update_q(obs[k], actions[k], rewards[k], next_obs[k], done)
                    total_reward[k] += rewards[k]
            obs = next_obs
            if render:
                rgb_array = env.render()
                surface = pygame.surfarray.make_surface(np.transpose(rgb_array, (1, 0, 2)))
                screen.blit(surface, (0, 0))
                pygame.display.flip()
                clock.tick(60)
            step_count += 1
            if render:
                for event in pygame.event.get():
                    if event.type == pygame.QUIT:
                        running = False

            if not running:
                break
        for _, agent in agents.items():
            agent.decay_epsilon(_ep)
    if render:
        pygame.quit()


In [13]:
run_episodes(episodes, steps)

Training: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 100/100 [00:45<00:00,  2.20ep/s]


In [14]:
run_episodes(1, 10000, True)

Training: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:02<00:00,  2.38s/ep]
